# コーディングをする前におこなったこと

1.　Europa Year Book の写真を撮る。

2.　Office Lensで Word形式にする。

3.　WordからWebノート形式に直す。

4.　3で直したWebノートをExcelで開き、行が国ごとに分かれていることを確認する。(シート名はReceiver Countryの名前)

5.　できたExcelをAzure NotebooksのLibraryにアップロードする(CCODE_PYTHON-ROWDATA.xlsx)。

6.　ccode-kurizaki-REVISED.xlsxを Libraryにアップロードする。

7.　New Coding Instruction for Diplomatic Representation Dataに従い、

　　ランクをエクセルに記入してAzure NotebooksのLibraryにアップロードする(SenderRank.xlsx)。



# 【１】必要なモジュールをインポートする

In [1]:
#コーディングで使うモジュールをインポートしていきます
# pandasのインポート
import pandas as pd
# numpyのインポート
import numpy as np
# 時間操作用ライブラリ
import time
from time import sleep
# 正規表現用reのインポート
import re
# ジオコーディング用ライブラリ
!pip install pygeocoder
from pygeocoder import Geocoder, GeocoderError
#日本語を含むコメントがはじかれないように
#coding: utf-8
#表示されるカラム内の文字数を80まで増やす(デフォルトは50だった)
pd.set_option("display.max_colwidth", 80)
#表示される行数を300まで増やす
pd.set_option("display.max_rows", 300)

# 【２】ccode-kurizaki-REVISED.xlsxに関して処理します
ccode-kurizaki-REVISED.xlsxを開く→データフレームにする→辞書型セットに直してSeries形式にするという流れです

In [2]:
#ccode-kurizaki.xlsxをinput_file_nameという名前のファイルにして操作しやすくします
input_file_name = 'ccode-kurizaki-REVISED.xlsx'
#このファイルをinput_bookとして開きます
input_book = pd.ExcelFile(input_file_name) 
#sheet_namesメソッドでExcelブック内の各シートの名前をリストで取得します
input_sheet_name = input_book.sheet_names

続いて、取得したリストをinput_sheet_dfという名前のデータフレームにして見やすくします

In [3]:
#input_sheet_name[0]は一枚目のシートを開くということです(二枚目のシートならinput_sheet_name[1]です)
#国番号(CCode)と国名(StateName)の列のみ取得するために、ccode-kurizaki.xlsxのB列からC列まで取得します
input_sheet_df = input_book.parse(input_sheet_name[0], 
                                  parse_cols 
                                  = "B:C",
                                  )
#列名(カラム名)をStateNameとCCodeとします
input_sheet_df = input_sheet_df[['StateName','CCode']]
#できたinput_sheet_dfの中身を見てみます
input_sheet_df

,StateName,CCode
0,United States of America,2.0
1,USA,2.0
2,Canada,20.0
3,Bahamas,31.0
4,Cuba,40.0
5,Cuba,40.0
6,Haiti,41.0
7,Haiti,41.0
8,Dominican Republic,42.0
9,Dominican Republic,42.0


このままだと後で取得した外交文書にある国名と比較しにくいので、このinput_sheet_dfをSeriesの形式に直します

In [4]:
#まずzipを使い、input_sheet_dfのStateNameにある国名、CCodeにある国番号の順にきれいな辞書型セットを作ります
CountryCode_dic = dict([(i,a) for i, a in zip(input_sheet_df.StateName, input_sheet_df.CCode)])
#CountryCode_dicの中身を見てみます
CountryCode_dic
#この辞書型データをSeriesに直します
CountryCode = pd.Series(CountryCode_dic)
CountryCode

Afghanistan                                               700.0
Albania                                                   339.0
Algeria                                                   615.0
Andorra                                                   232.0
Angola                                                    540.0
Antigua & Barbuda                                          58.0
Argentina                                                 160.0
Armenia                                                   371.0
Australia                                                 900.0
Austria                                                   305.0
Austria-Hungary                                           300.0
Azerbaijan                                                373.0
Baden                                                     267.0
Bahamas                                                    31.0
Bahrain                                                   692.0
Bangladesh                              

input_sheet_dfとCountryCodeで行(row)の数が違うのは、input_sheet_dfでは重複するものがあるからです(Japanが二つなど)

ここでいったんccode-kurizaki.xlsxに関する準備は終わります

# 【３】SenderRank.xlsxに関して処理していきます
SenderRank.xlsxを開く→データフレームにするという流れです

In [5]:
#以下ccode-kurizaki.xlsxのときと同じ手順です
#今回はファイルの名前をinput_file_name2にします
input_file_name2 = 'SenderRank.xlsx'
#input_book2として開きます
input_book2 = pd.ExcelFile(input_file_name2) 
#sheet_namesメソッドでExcelブック内の各シートの名前をinput_sheet_name2としてリストで取得します
input_sheet_name2 = input_book2.sheet_names
#dataフレームとしてinput_sheet_name2の一つ目のsheetを読み込みます（input_sheet_name2[0]）
input_sheet_df2 = input_book2.parse(input_sheet_name2[0])
#input_sheet_df2の中身を見てみます
input_sheet_df2

,Rank,Code
0,Not assigned or appointed,0
1,Ambassador,1
2,Nuncio,1
3,Apostolic nuncio,1
4,Apostolic Nuncio,1
5,papal nuncio,1
6,Papal nuncio,1
7,Papal Nuncio,1
8,pro-nuncio,1
9,Pro nuncio,1


SenderRank.xlsxに関する準備は以上です

# 【４】それではCCode_PYTHON-ROWDATA.xlsxを取り込みます

<１>　文字列の形式を整える

①このExcelを開き、文字列を;や:で分割します(この時先頭のいらない空白を要素ごとに削除します)--->df2

②df2のうち、数字が入っている要素はNaNとし全ての行にNanが表示されている列を削除します--->df2_copied

<２>　国と国番号

df2_copiedから国名の部分だけ取り出し、対応する国番号をふって一列作ります--->df3


<３>　SenderRankとSenderRankCode

df2_copiedの要素の中でSenderRankのリストにあるRank名があれば取り出し、対応する番号をふって一列作ります--->df4


<４>　SAとSAH

①住所欄の情報を","や空白で分割し、数字の要素などを消したうえで都市のみをリスト化します--->City

②geocorderでその都市の緯度・経度情報を出し、属している国を割り出します--->find_country関数

③割り出した国とReceiverCountryが一致すればSAの欄には0,SAHには7を記入し、一致しなければSAに1,SAHにはその割り出した国の国番号を記入--->df5


<５>　SenderNameとVA

①df2_copiedのコピーdf2_copied2を作り、このText4の部分をSenderNameを取り出した列に置き換えてます--->df6"SenderName"

②df6のSenderNameを用いて、名前があればVacant=7とします

<６>　<２>～<５>で作ったデータフレーム(df3～df6)の結合と取り出されなかった要素

①df3～df6の結合--->organized

②②このorganizedに、ReceiverCountryやReceiverCode, Volume, Yearなどを追加していきます--->organized

③df2_copiedの要素の中でorganizedにない要素も再登場させます--->df7


## <１>　文字列の形式を整える

まずは①の作業です

Excel「CCODE_PYTHON-ROWDATA.xlsx」を読み込みます

(写真をスキャンした生のデータで、REVISEDと違い行の最初に国名を記入せず、空白の削除や:を;に変えることもしていません)

In [6]:
#以下ccode-kurizaki.xlsxのときと同じ手順です
#今回はファイルの名前をinput_file_name3にします
input_file_name3 = 'CCODE_PYTHON-ROWDATA.xlsx'
#input_book3として開きます
input_book3 = pd.ExcelFile(input_file_name3) 
#sheet_namesメソッドでExcelブック内の各シートの名前をinput_sheet_name3としてリストで取得します
input_sheet_name3 = input_book3.sheet_names
#lenでinput_sheet_name3のシートの総数を確認します
num_sheet = len(input_sheet_name3)
#input_sheet_name3のシートの数とシートの名前のリストの表示
print ("Sheet の数:", num_sheet)
print (input_sheet_name3)

Sheet の数: 1
['Kazakhstan']


In [7]:
#dataフレームとしてinput_sheet_name3の一つ目のsheetを読み込みます（input_sheet_name3[0]）
#header = Noneは、元のExcelで列名(カラム名)を設けていない場合、最初の行が列名だと認識されるのを防ぐためにおきます
input_sheet_df3 = input_book3.parse(input_sheet_name3[0],header=None)
#読み込んだシート名の確認します
print("Sheet name:", input_sheet_name3[0])
#できたデータフレームを確認します
input_sheet_df3 

Sheet name: Kazakhstan


,0,1,2,3,4,5,6,7,8,9
0,"Afghanistan: Almaty, Shalyapina 56, Hotel Molodezhnaya; tel. (3272) 28-68-71...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Armenia: 480075 Almaty, pr. Seifullina 579, 7th Floor; tel. and fax (3272) 6...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Australia: 480004 Almaty, Kazybek bi 20A; tel. (3272) 63-94-18;Ambassador: D...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Bulgaria: Almaty, Makataeva 13A; tel. (3272) 30-27-55;Chargé d'affaires a.i....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Canada: 480100 Almaty, Karasai Batyr 34; tel. (3272) 50-11-51;Ambassador: CH...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"China, People's Republic: Almaty, Furmanova 137 ; tel. (3272) 63-49-66; fax ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"Croatia: Almaty, Furmanova 110; tel. (3272) 62-57-03; telex 251435; fax (327...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"Cuba: Almaty, Zenkova 70, kv. 23; tel. and fax (3272) 61-59-25;Chargé d'affa...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"Czech Republic: Almaty, pr. Zhibek zholy 64; tel. (3272) 33-47-13; fax (3272...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"Egypt: 480100 Almaty, Zenkova 59; tel. (3272) 60-16-22; fax (3272) 61-10-22;...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#1, 2 ,,,という列はいらないので消去します
#parse_cols = 0は一列目だけとるという意味です(二列目だけをとるならparse_cols = 1)header = Noneを入れています
#ここでも列名を設けていないため、header = Noneをおきます
input_sheet_df3 = input_book3.parse(input_sheet_name3[0], 
                                  parse_cols = 0,
                                  header=None  
                                  )

input_sheet_df3

,0
0,"Afghanistan: Almaty, Shalyapina 56, Hotel Molodezhnaya; tel. (3272) 28-68-71..."
1,"Armenia: 480075 Almaty, pr. Seifullina 579, 7th Floor; tel. and fax (3272) 6..."
2,"Australia: 480004 Almaty, Kazybek bi 20A; tel. (3272) 63-94-18;Ambassador: D..."
3,"Bulgaria: Almaty, Makataeva 13A; tel. (3272) 30-27-55;Chargé d'affaires a.i...."
4,"Canada: 480100 Almaty, Karasai Batyr 34; tel. (3272) 50-11-51;Ambassador: CH..."
5,"China, People's Republic: Almaty, Furmanova 137 ; tel. (3272) 63-49-66; fax ..."
6,"Croatia: Almaty, Furmanova 110; tel. (3272) 62-57-03; telex 251435; fax (327..."
7,"Cuba: Almaty, Zenkova 70, kv. 23; tel. and fax (3272) 61-59-25;Chargé d'affa..."
8,"Czech Republic: Almaty, pr. Zhibek zholy 64; tel. (3272) 33-47-13; fax (3272..."
9,"Egypt: 480100 Almaty, Zenkova 59; tel. (3272) 60-16-22; fax (3272) 61-10-22;..."


ここから文字列を:や;で分割していきます

In [9]:
#このinput_sheet_df3をコピーしたものをdfとします(コピーなのでinput_sheet_df3と同じ内容です)
df = input_sheet_df3.copy()
#iloc[:, 0]で「全ての行、0列目」を;か:で分割します
df2 = df.iloc[:, 0].str.split(';|:', expand=True)
#str.stripを使い0,1,2,3,4,5,6列目の要素の先頭にある空白をなくします
df2 = df2.apply(lambda d: d.str.strip())
#列名を"Text1","Text2","Text3","Text4","Text5","Text6","Text7"とします
df2.columns = ["Text1","Text2","Text3","Text4","Text5","Text6","Text7"]
df2

,Text1,Text2,Text3,Text4,Text5,Text6,Text7
0,Afghanistan,"Almaty, Shalyapina 56, Hotel Molodezhnaya",tel. (3272) 28-68-71,Chargé d'affaires a.i.,NAJIBULLAH ZIA RAKHMAN.,None,None
1,Armenia,"480075 Almaty, pr. Seifullina 579, 7th Floor",tel. and fax (3272) 69-29-08,Chargé d’affaires a.i.,ARMAN MELIKIAN.,None,None
2,Australia,"480004 Almaty, Kazybek bi 20A",tel. (3272) 63-94-18,Ambassador,DOUGLAS TOWNSEND.,None,None
3,Bulgaria,"Almaty, Makataeva 13A",tel. (3272) 30-27-55,Chargé d'affaires a.i.,YORDAN TRENCHEV.,None,None
4,Canada,"480100 Almaty, Karasai Batyr 34",tel. (3272) 50-11-51,Ambassador,CHARLES RICHARD MANN.,None,None
5,"China, People's Republic","Almaty, Furmanova 137",tel. (3272) 63-49-66,fax (3272) 63-92-91,Ambassador,CHEN DI.,None
6,Croatia,"Almaty, Furmanova 110",tel. (3272) 62-57-03,telex 251435,fax (3272) 50-62-92,Chargé d'affaires a.i.,KARINO HROMIN.
7,Cuba,"Almaty, Zenkova 70, kv. 23",tel. and fax (3272) 61-59-25,Chargé d'affaires a.i.,ROBERTO CRUZ GALINDO.,None,None
8,Czech Republic,"Almaty, pr. Zhibek zholy 64",tel. (3272) 33-47-13,fax (3272) 33-50-88,Chargé d'affaires a.i.,ALEXANDER LANGER.,None
9,Egypt,"480100 Almaty, Zenkova 59",tel. (3272) 60-16-22,fax (3272) 61-10-22,Ambassador,AYMAN HAMDI EL KOUNI.,None


これで①の分割作業は終了です

## 続いて、全行に数字のある列を削除する②の作業に移ります


In [10]:
#df2の内容を複製したものをdf2_copiedとします
df2_copied = df2.copy()
#df2の要素の中でtel,fax,telex,(3272),数字が入っている要素を削除します
#削除した後のデータフレームはretという名前です
ret = df2_copied.apply(lambda d: d.str.contains('tel|fax|(3272)|telex|pr|\d', na=False, regex=True))
df2_copied = df2_copied.mask(ret)
#df2_opiedの中身を見てみます
df2_copied

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


,Text1,Text2,Text3,Text4,Text5,Text6,Text7
0,Afghanistan,NaN,NaN,Chargé d'affaires a.i.,NAJIBULLAH ZIA RAKHMAN.,None,None
1,Armenia,NaN,NaN,Chargé d’affaires a.i.,ARMAN MELIKIAN.,None,None
2,Australia,NaN,NaN,Ambassador,DOUGLAS TOWNSEND.,None,None
3,Bulgaria,NaN,NaN,Chargé d'affaires a.i.,YORDAN TRENCHEV.,None,None
4,Canada,NaN,NaN,Ambassador,CHARLES RICHARD MANN.,None,None
5,"China, People's Republic",NaN,NaN,NaN,Ambassador,CHEN DI.,None
6,Croatia,NaN,NaN,NaN,NaN,Chargé d'affaires a.i.,KARINO HROMIN.
7,Cuba,NaN,NaN,Chargé d'affaires a.i.,ROBERTO CRUZ GALINDO.,None,None
8,Czech Republic,NaN,NaN,NaN,Chargé d'affaires a.i.,ALEXANDER LANGER.,None
9,Egypt,NaN,NaN,NaN,Ambassador,AYMAN HAMDI EL KOUNI.,None


In [11]:
#全ての行にNaNと入っている列を消します
df2_copied = df2_copied.dropna(axis = 1, how = 'all')
#df2の中身を見てみます
df2_copied

,Text1,Text4,Text5,Text6,Text7
0,Afghanistan,Chargé d'affaires a.i.,NAJIBULLAH ZIA RAKHMAN.,None,None
1,Armenia,Chargé d’affaires a.i.,ARMAN MELIKIAN.,None,None
2,Australia,Ambassador,DOUGLAS TOWNSEND.,None,None
3,Bulgaria,Chargé d'affaires a.i.,YORDAN TRENCHEV.,None,None
4,Canada,Ambassador,CHARLES RICHARD MANN.,None,None
5,"China, People's Republic",NaN,Ambassador,CHEN DI.,None
6,Croatia,NaN,NaN,Chargé d'affaires a.i.,KARINO HROMIN.
7,Cuba,Chargé d'affaires a.i.,ROBERTO CRUZ GALINDO.,None,None
8,Czech Republic,NaN,Chargé d'affaires a.i.,ALEXANDER LANGER.,None
9,Egypt,NaN,Ambassador,AYMAN HAMDI EL KOUNI.,None


②の作業は以上です(ここまではROWDATAを操作可能な形式にしただけ)

# <２>　国と国番号

## ではdf2_copiedのText1にある国に番号をふった列を作ります

まず、df2_copiedのText1にある国名だけをSeriesにします

In [12]:
#まずdf2のCountrnameという列にある国名をCountrylistというきれいなリストにします
Countrylist = df2_copied.iloc[:, 0].values.tolist()
#Series形式に直します
Country = pd.Series(Countrylist)
#Countryの中身を見てみます
Country

0                             Afghanistan
1                                 Armenia
2                               Australia
3                                Bulgaria
4                                  Canada
5                China, People's Republic
6                                 Croatia
7                                    Cuba
8                          Czech Republic
9                                   Egypt
10                                 France
11                                Georgia
12                                Germany
13                               Holy See
14                                Hungary
15                                  India
16                                   Iran
17                                 Israel
18                                  Italy
19                                  Japan
20    Korea, Democratic People's Republic
21                        Korea, Republic
22                             Kyrgyzstan
23                                

これらの表示された国名と国番号をまとめたデータフレームをdf3とします

In [13]:
#CountryCodeとCountryを比べdf3に格納します
df3 = CountryCode[CountryCode.index.isin(Country)]
#整理し、列名をSenderCountry, SenderCodeとします
df3 = df3.reset_index().rename(columns={'index':'SenderCountry'})
df3.columns = ['SenderCountry', 'SenderCode']
#df3の中身を見てみます
df3

,SenderCountry,SenderCode
0,Afghanistan,700.0
1,Armenia,371.0
2,Australia,900.0
3,Bulgaria,355.0
4,Canada,20.0
5,"China, People's Republic",710.0
6,Croatia,344.0
7,Cuba,40.0
8,Czech Republic,316.0
9,Egypt,651.0


読み取った全ての国に国番号が示されました

ここまでが<２>の作業です

## <３>では国番号と同じように、Rankとそれに対応する番号の列も作ります

df2_copiedのコピーをdf4とし、ここにRankごとに番号をふった列を付け加えていきます

In [14]:
#df2_copiedのコピーをdf4とします
df4 = df2_copied.copy()
# df4 より Rank名->数値の変換用のSeriesデータを作成します
conv = input_sheet_df2.set_index('Rank').Code
# 行毎にループ(df4の行と列において)
for idx, row in df4.iterrows():
    # Text1以降の列を変換データにて変換(input_sheet_df2のRank名に該当するものがあれば抽出)
    tmp = row["Text1":].map(conv)
    # 変換できたデータ(Rank名のみ)を'SenderRank'列に追加
    df4.loc[idx, 'SenderRank'] = row[row['Text1':].index[tmp.notnull()][0]]
    # 上の処理で変換できなかったデータは削除
    tmp = tmp.dropna()
    #NaNでなければ表示するという条件を作ります
    if tmp.size > 0:
        # ふられた番号はSenderRankCodeという列にいれていきます
        df4.loc[idx, 'SenderRankCode'] = tmp[0]
        
#できたdf4の中身を見てみます        
df4

,Text1,Text4,Text5,Text6,Text7,SenderRank,SenderRankCode
0,Afghanistan,Chargé d'affaires a.i.,NAJIBULLAH ZIA RAKHMAN.,None,None,Chargé d'affaires a.i.,3.0
1,Armenia,Chargé d’affaires a.i.,ARMAN MELIKIAN.,None,None,Chargé d’affaires a.i.,3.0
2,Australia,Ambassador,DOUGLAS TOWNSEND.,None,None,Ambassador,1.0
3,Bulgaria,Chargé d'affaires a.i.,YORDAN TRENCHEV.,None,None,Chargé d'affaires a.i.,3.0
4,Canada,Ambassador,CHARLES RICHARD MANN.,None,None,Ambassador,1.0
5,"China, People's Republic",NaN,Ambassador,CHEN DI.,None,Ambassador,1.0
6,Croatia,NaN,NaN,Chargé d'affaires a.i.,KARINO HROMIN.,Chargé d'affaires a.i.,3.0
7,Cuba,Chargé d'affaires a.i.,ROBERTO CRUZ GALINDO.,None,None,Chargé d'affaires a.i.,3.0
8,Czech Republic,NaN,Chargé d'affaires a.i.,ALEXANDER LANGER.,None,Chargé d'affaires a.i.,3.0
9,Egypt,NaN,Ambassador,AYMAN HAMDI EL KOUNI.,None,Ambassador,1.0


In [15]:
#いらないText4～Text7までの列を削除し、新たに作ったSenderRankとSenderRankCodeの列のみ残します
df4 = df4.drop(['Text4','Text5','Text6','Text7'], axis=1)
#変更したdf4の中身を確認します
df4

,Text1,SenderRank,SenderRankCode
0,Afghanistan,Chargé d'affaires a.i.,3.0
1,Armenia,Chargé d’affaires a.i.,3.0
2,Australia,Ambassador,1.0
3,Bulgaria,Chargé d'affaires a.i.,3.0
4,Canada,Ambassador,1.0
5,"China, People's Republic",Ambassador,1.0
6,Croatia,Chargé d'affaires a.i.,3.0
7,Cuba,Chargé d'affaires a.i.,3.0
8,Czech Republic,Chargé d'affaires a.i.,3.0
9,Egypt,Ambassador,1.0


以上で<３>の作業は終わります

# <４>　SAについて処理していきます

SAの判断のために、住所に書いてある場所がどこの国に属しているのかを見ます

①その際、住所欄の情報を","や空白で分割し、数字の要素などを消したうえで都市のみをリスト化します

②geocorderでその都市の緯度・経度情報を出し、属している国を割り出します(ResidenceのCountryという列)

↓

続いて、割り出した国とReceiverCountryとを比べます

③割り出した国とReceiverCountryが一致すればSAの欄には0,SAHには7を記入し、一致しなければSAに1,SAHにはその割り出した国の国番号を記入します

それでは、①の操作に入ります

In [16]:
#まず、df4の'Text1','Text2'だけとりだしたデータフレームdf5を作ります
ADDRESS = df2[['Text1', 'Text2']]
# ADDRESSからSenderCountryの列のみ削除します
ADDRESS = ADDRESS.drop(['Text1'], axis=1)
#iloc[:, 0]で「全ての行、0列目」を,か空白で分割します
ADDRESS = ADDRESS.iloc[:, 0].str.split(',| ', expand=True)
#ADDRESSの要素の中でpr.,数字が入っている要素を削除します
#削除した後のデータフレームはdroppedという名前です
dropped = ADDRESS.apply(lambda d: d.str.contains('pr|\d', na=False, regex=True))
ADDRESS = ADDRESS.mask(dropped)
# NaNと表示されているところは左詰めにします
ADDRESS = ADDRESS.fillna(method='bfill',axis=1)
ADDRESS

,0,1,2,3,4,5,6,7,8
0,Almaty,,Shalyapina,,,Hotel,Molodezhnaya,None,None
1,Almaty,Almaty,,Seifullina,Seifullina,,,Floor,Floor
2,Almaty,Almaty,,Kazybek,bi,None,None,None,None
3,Almaty,,Makataeva,None,None,None,None,None,None
4,Almaty,Almaty,,Karasai,Batyr,None,None,None,None
5,Almaty,,Furmanova,None,None,None,None,None,None
6,Almaty,,Furmanova,None,None,None,None,None,None
7,Almaty,,Zenkova,,,kv.,None,None,None
8,Almaty,,Zhibek,Zhibek,zholy,None,None,None,None
9,Almaty,Almaty,,Zenkova,None,None,None,None,None


In [17]:
# ADDRESSの一列目だけをCityとして取り出します
City = ADDRESS.iloc[:,0]
# Cityをデータフレーム化します
City = pd.DataFrame(City)
# Cityの列名をCityとします
City.columns = {"City"}
# Cityの中身を見てみます
City

,City
0,Almaty
1,Almaty
2,Almaty
3,Almaty
4,Almaty
5,Almaty
6,Almaty
7,Almaty
8,Almaty
9,Almaty


ここまでで①の作業は終わります

次に、②の操作でこのCityの緯度・経度情報から属している国を割り出します

In [18]:
# SAやSAH用のデータフレームをdf5とします
df5 = pd.DataFrame()
# Geocoderというライブラリを使って都市名から国を割り出します
#まず、都市名から緯度・経度情報を割り出す関数を作ります
def find_country(name):
    geocode = Geocoder.geocode(name)
    address_components = geocode.current_data["address_components"]
    #発見したアドレスから国名だけを取り出すようにします
    for d in address_components:
        if "country" in d["types"]:
            return d["long_name"]

#countriesという空のリストを作ります        
countries = []
#先程のCityというデータフレームの都市において find_countryの関数を適用します(while Trueで見つかったもののみ出します)
for city in City["City"]:
    while True:
        try:
            country = find_country(city)
            print(country)
            countries.append(country)
            time.sleep(1)
            break
        except GeocoderError:
            time.sleep(1)
            continue
#できたcountriesというリストをdf５の"Residence_Country"という列の中に組み込みます
df5["Residence_Country"] = countries
#df５の中身を見てみます
df5

Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan
Kazakhstan


,Residence_Country
0,Kazakhstan
1,Kazakhstan
2,Kazakhstan
3,Kazakhstan
4,Kazakhstan
5,Kazakhstan
6,Kazakhstan
7,Kazakhstan
8,Kazakhstan
9,Kazakhstan


In [19]:
# SA, SAHという空のリストを作ります
SA = []
SAH = []
#df5の行に対し
for rows in df5.itertuples():
    #さらにその行の中の要素に対し
    for items in rows:
        #input_sheet_name3[0]と同じならば
        if items == input_sheet_name3[0]:
            #SAには0、SAHには7を付け加え、異なるならばSAには1,SAHには出てきた国の国番号を記入する(CountryCode_dicより)という条件を作ります
            SA.append(0)
            SAH.append(7)
            break
    else:
        SA.append(1)
        SAH.append(CountryCode_dic[items])
#df5に先程の作ったSA,SAHという列を入れる用の列を作り、SA,SAHというカラム名を付けます
df5["SA"] = SA
df5["SAH"] = SAH

#df5の中身を見てみます
df5

,Residence_Country,SA,SAH
0,Kazakhstan,0,7
1,Kazakhstan,0,7
2,Kazakhstan,0,7
3,Kazakhstan,0,7
4,Kazakhstan,0,7
5,Kazakhstan,0,7
6,Kazakhstan,0,7
7,Kazakhstan,0,7
8,Kazakhstan,0,7
9,Kazakhstan,0,7


これで<４>の作業を終わります

## <５>では、SenderNameを用いて、VAを求めていきます

①df2_copiedのコピーdf2_copied2を作り、このText4の部分をSenderNameを取り出した列に置き換えて新たにdf6を作ります

In [21]:
# df2_copiedのコピーdf2_copied2を作ります
df2_copied2 = df2_copied.copy()
# このままだとUSAなどの情報が記録されてしまうのでText1の列を削除します
df2_copied2 = df2_copied2.drop(['Text1'], axis=1)
# 正規表現reを用いて、大文字で始まり大文字で終わっている文字列のコンパイルをcapitalとセットしておきます
capital = re.compile(r"\b[A-Z]+\b")
# capitalsという関数を作ります(先程設定した"capital"の条件と合う要素があればそれを取り出していくという関数です)
def capitals(item):
    ans = capital.findall(item)
    if len(ans) == 0:
        return ''
    else:
        return ' '.join(ans)
# df2_copied2にcapitals関数を適用したデータフレームをtdfとします
tdf = df2_copied2.astype(str).applymap(capitals)
# tdfの配列を正します
tdf = tdf.apply(sorted, axis=1, reverse=True)
df6 = tdf.dropna(how = "all" ,axis = 1)
#Text4の列名をSenderNameに変更します
df6.rename(columns={"Text4" : "SenderName"}, inplace=True)
#悲痛ようのない"Text5","Text6","Text7"の列を削除します
df6 = df6.drop(["Text5","Text6","Text7"], axis = 1)
#df6の中身を確認します
df6

,SenderName
0,NAJIBULLAH ZIA RAKHMAN
1,ARMAN MELIKIAN
2,DOUGLAS TOWNSEND
3,YORDAN TRENCHEV
4,CHARLES RICHARD MANN
5,CHEN DI
6,KARINO HROMIN
7,ROBERTO CRUZ GALINDO
8,ALEXANDER LANGER
9,AYMAN HAMDI EL KOUNI


②df6のSenderNameを用いて、名前があればVacant=0とします

In [22]:
#Vacantという空のリストを作ります
Vacant = []
#df6の行に対し
for index in df6.itertuples():
    #さらにその行の中の要素に対し
    for elements in index:
        #要素が空であれば
        if elements == None:
            #Vacantには1を付け加えるという条件を作ります
            Vacant.append(1)
            break
    else:
        Vacant.append(7)

#df6に先程の作ったVacantを入れる用の列を作り、Vacantというカラム名を付けます
df6["VA"] = Vacant
#df6の中身を見てみます
df6

,SenderName,VA
0,NAJIBULLAH ZIA RAKHMAN,7
1,ARMAN MELIKIAN,7
2,DOUGLAS TOWNSEND,7
3,YORDAN TRENCHEV,7
4,CHARLES RICHARD MANN,7
5,CHEN DI,7
6,KARINO HROMIN,7
7,ROBERTO CRUZ GALINDO,7
8,ALEXANDER LANGER,7
9,AYMAN HAMDI EL KOUNI,7


これで<５>の作業は終わりです

# それでは<６>でdf3~df6を統合します

まず、①で今まで取り出した要素の列をつなげて一つのデータフレームorganizedを作ります

In [23]:
#SenderCountryとSenderCodeのあるdf3、df4のSenderRankとSenderRankCode,df5のResidence_CountryとSA,SAH, そしてdf6のSenderNameとVAをつなげます
#新しいデータフレームをorganizedとします
organized = pd.concat([df3,df4['SenderRank'],df4['SenderRankCode'],df5['Residence_Country'],df5['SA'],df5['SAH'],df6['SenderName'],df6['VA']], axis=1)
# organizedの中身を確認します
organized 

,SenderCountry,SenderCode,SenderRank,SenderRankCode,Residence_Country,SA,SAH,SenderName,VA
0,Afghanistan,700.0,Chargé d'affaires a.i.,3.0,Kazakhstan,0,7,NAJIBULLAH ZIA RAKHMAN,7
1,Armenia,371.0,Chargé d’affaires a.i.,3.0,Kazakhstan,0,7,ARMAN MELIKIAN,7
2,Australia,900.0,Ambassador,1.0,Kazakhstan,0,7,DOUGLAS TOWNSEND,7
3,Bulgaria,355.0,Chargé d'affaires a.i.,3.0,Kazakhstan,0,7,YORDAN TRENCHEV,7
4,Canada,20.0,Ambassador,1.0,Kazakhstan,0,7,CHARLES RICHARD MANN,7
5,"China, People's Republic",710.0,Ambassador,1.0,Kazakhstan,0,7,CHEN DI,7
6,Croatia,344.0,Chargé d'affaires a.i.,3.0,Kazakhstan,0,7,KARINO HROMIN,7
7,Cuba,40.0,Chargé d'affaires a.i.,3.0,Kazakhstan,0,7,ROBERTO CRUZ GALINDO,7
8,Czech Republic,316.0,Chargé d'affaires a.i.,3.0,Kazakhstan,0,7,ALEXANDER LANGER,7
9,Egypt,651.0,Ambassador,1.0,Kazakhstan,0,7,AYMAN HAMDI EL KOUNI,7


②このorganizedに、ReceiverCountryやReceiverCode, Volume, Yearなどを追加していきます

In [24]:
#ReceiverCountryには、国名を記入しますが、Excelのシート名でもあるので以前定義したように記します
organized['ReceiverCountry']= input_sheet_name3[0]
#ReceiverCodeには、ReceiverCountryの国番号を自動的に出すようにします
organized['ReceiverCode'] = CountryCode_dic[input_sheet_name3[0]]
#私が今回用いたEuropa Year Bookの年・巻番号を記しました
organized['Volume_Year'] = "1997Ⅱ"
#organizedの中身を見てみます
organized

,SenderCountry,SenderCode,SenderRank,SenderRankCode,Residence_Country,SA,SAH,SenderName,VA,ReceiverCountry,ReceiverCode,Volume_Year
0,Afghanistan,700.0,Chargé d'affaires a.i.,3.0,Kazakhstan,0,7,NAJIBULLAH ZIA RAKHMAN,7,Kazakhstan,705.0,1997Ⅱ
1,Armenia,371.0,Chargé d’affaires a.i.,3.0,Kazakhstan,0,7,ARMAN MELIKIAN,7,Kazakhstan,705.0,1997Ⅱ
2,Australia,900.0,Ambassador,1.0,Kazakhstan,0,7,DOUGLAS TOWNSEND,7,Kazakhstan,705.0,1997Ⅱ
3,Bulgaria,355.0,Chargé d'affaires a.i.,3.0,Kazakhstan,0,7,YORDAN TRENCHEV,7,Kazakhstan,705.0,1997Ⅱ
4,Canada,20.0,Ambassador,1.0,Kazakhstan,0,7,CHARLES RICHARD MANN,7,Kazakhstan,705.0,1997Ⅱ
5,"China, People's Republic",710.0,Ambassador,1.0,Kazakhstan,0,7,CHEN DI,7,Kazakhstan,705.0,1997Ⅱ
6,Croatia,344.0,Chargé d'affaires a.i.,3.0,Kazakhstan,0,7,KARINO HROMIN,7,Kazakhstan,705.0,1997Ⅱ
7,Cuba,40.0,Chargé d'affaires a.i.,3.0,Kazakhstan,0,7,ROBERTO CRUZ GALINDO,7,Kazakhstan,705.0,1997Ⅱ
8,Czech Republic,316.0,Chargé d'affaires a.i.,3.0,Kazakhstan,0,7,ALEXANDER LANGER,7,Kazakhstan,705.0,1997Ⅱ
9,Egypt,651.0,Ambassador,1.0,Kazakhstan,0,7,AYMAN HAMDI EL KOUNI,7,Kazakhstan,705.0,1997Ⅱ


③organizedとdf2_copiedの中身を比べ、df2にあってorganizedにはない要素があれば新しい列に加えてdf6を作ります

In [25]:
# Errorを防ぐため、df2_copiedの国名の列を一度削除します
df2_copied = df2_copied.drop(['Text1'], axis=1)
# organizedとdf2_copiedの中身を比べ、重複しているものを消していきます
df7 = pd.concat([organized, df2_copied], axis=1).apply(lambda d: d.drop_duplicates(inplace= False).reset_index(drop=True), axis=1)
#全ての行にNaNと表示されている列を削除します
df7 = df7.dropna(axis = 1, how = 'all')
#列名を"SenderCountry","SenderCode","SenderRank","SenderRankCode","Address","SenderName","ReceiverCountry","ReceiverCode","Year/Volume","Others1","Others2","Others3"とします
df7.columns = ["SenderCountry","SenderCode","SenderRank","SenderRankCode","Address","SenderName","ReceiverCountry","ReceiverCode","Year/Volume","Others1","Others2","Others3"]
#このままだと"Others1","Others2","Others3"の中に"."が入っているSenderNameが表示されていしまうので、大文字で書かれた要素は消すDropSenderという関数を作ります
DropSender = lambda x: 'None' if hasattr(x, "isupper") and x.isupper() else x
#df7の"Others1","Others2","Others3"に対しこのDropSenderを適用したOthersChangedというデータフレームを作ります
OthersChanged = df7[["Others1","Others2","Others3"]].applymap(DropSender)
#一度df7から"修正していないOthers1","Others2","Others3"を削除し
df7 = df7.drop(["Others1", "Others2","Others3"],axis=1)
#OthersChangedと入れ替えます
df7 = pd.concat([df7, OthersChanged], axis=1)
#できたdf7を確認します
df7

,SenderCountry,SenderCode,SenderRank,SenderRankCode,Address,SenderName,ReceiverCountry,ReceiverCode,Year/Volume,Others1,Others2,Others3
0,Afghanistan,700.0,Chargé d'affaires a.i.,3.0,Kazakhstan,0,7,NAJIBULLAH ZIA RAKHMAN,705.0,None,None,None
1,Armenia,371.0,Chargé d’affaires a.i.,3.0,Kazakhstan,0,7,ARMAN MELIKIAN,705.0,None,None,None
2,Australia,900.0,Ambassador,1.0,Kazakhstan,0,7,DOUGLAS TOWNSEND,705.0,None,None,None
3,Bulgaria,355.0,Chargé d'affaires a.i.,3.0,Kazakhstan,0,7,YORDAN TRENCHEV,705.0,None,None,None
4,Canada,20.0,Ambassador,1.0,Kazakhstan,0,7,CHARLES RICHARD MANN,705.0,None,None,None
5,"China, People's Republic",710.0,Ambassador,1.0,Kazakhstan,0,7,CHEN DI,705.0,None,NaN,None
6,Croatia,344.0,Chargé d'affaires a.i.,3.0,Kazakhstan,0,7,KARINO HROMIN,705.0,None,NaN,None
7,Cuba,40.0,Chargé d'affaires a.i.,3.0,Kazakhstan,0,7,ROBERTO CRUZ GALINDO,705.0,None,None,None
8,Czech Republic,316.0,Chargé d'affaires a.i.,3.0,Kazakhstan,0,7,ALEXANDER LANGER,705.0,None,NaN,None
9,Egypt,651.0,Ambassador,1.0,Kazakhstan,0,7,AYMAN HAMDI EL KOUNI,705.0,None,NaN,None


これで③の作業は終わります

# 【5】完成したデータフレームdf6をCSVに保存します

In [26]:
#pandas形式の場合.to_csvというメソッドでCSV化できます
#Excelで開きたい場合、encoding="SHIFT-JIS"という引数を追加してSHIFT-JISでエンコーディングするのが最適です
df7.to_csv("CCode2.csv", index=False, encoding="'Shift_JISx0213'")

↑を実行して、Libraryの中に入っていればcsvとして保存できています(Libraryのページを開きなおしてください)